In [1]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

from datetime import datetime
import re
from collections import defaultdict
import json

from crawler_model import Ptt

ua = UserAgent()
fakeua = ua.random

In [2]:
# Read
with open('phone_catalog.json', 'r') as fp:
    phone_dict = json.load(fp)

In [ ]:
url = "https://en.wikipedia.org/wiki/List_of_Android_smartphones"

# GET request from url and parse via BeautifulSoup
resp = requests.get(url)
#resp.encoding = 'utf-8' # encoded with format utf-8 for chinese character
soup = BeautifulSoup(resp.text, "html.parser")

In [ ]:
tables = soup.find_all("table", class_="wikitable")
rows = []
for i in tables:
    rows.extend(i.select("tbody tr th"))

In [ ]:
phones = [th.text.partition("(")[0].strip() for th in rows]
phones[5]

In [ ]:
brands = set()
for p in phones:
    brands.add(p.split(" ")[0])
brands = list(brands)
brands = sorted(brands)
for b in range(len(brands)):
    if brands[b] == "Asus":
        brands = brands[b:]
        break

In [ ]:
phone_dict = defaultdict(list)
for b in brands:
    if b == "Samsung":
        for p in phones:
            if p.split(" ")[0] == b:
                phone = p.replace(b + " Galaxy", "").replace("5G", "").strip()
                if len(phone) > 0:
                    phone_dict[b].append(phone)
    elif b == "POCO":
        for p in phones:
            if p.split(" ")[0] == b:
                phone = p.replace("5G", "").strip()
                if len(phone) > 0:
                    phone_dict["Xiaomi"].append(phone)
    else:
        for p in phones:
            if p.split(" ")[0] == b:
                phone = p.replace(b, "").replace("5G", "").strip()
                if len(phone) > 0:
                    phone_dict[b].append(phone)

# phone_dict

## Android

In [ ]:
url = "https://en.wikipedia.org/wiki/List_of_Android_smartphones"
resp = requests.get(url)
soup = BeautifulSoup(resp.text, "html.parser")

tables = soup.find_all("table", class_="wikitable")
rows = []
for i in tables:
    rows.extend(i.select("tbody tr th"))

phones = [a.text.partition("(")[0].strip() for a in rows]
phones = [ p for p in phones if len(p.split()) > 1]

brands = set()
for p in phones:
    brands.add(p.split(" ")[0])
brands = list(brands)
brands = sorted(brands)
for b in range(len(brands)):
    if brands[b] == "Asus":
        brands = brands[b:]
        break

phone_dict = defaultdict(list)
for b in brands:
    if b == "Samsung":
        for p in phones:
            if p.split(" ")[0] == b:
                if "/" in p:
                    p_lst = p.split("/")
                    for series in p_lst[1:]:
                        sub_p = p_lst[0].replace(b + " Galaxy", "").replace("5G", "").strip()
                        if series == "+":
                            phone = sub_p + series
                        else:
                            phone = sub_p + " " + series
                        if len(phone) > 0:
                            phone_dict[b].append(phone)
                    phone_dict[b].append(sub_p)
                else:
                    phone = p.replace(b + " Galaxy", "").replace("5G", "").strip()
                    if len(phone) > 0:
                        phone_dict[b].append(phone)
    elif b == "POCO":
        for p in phones:
            if p.split(" ")[0] == b:
                phone = p.replace("5G", "").strip()
                if len(phone) > 0:
                    phone_dict["Xiaomi"].append(phone)
    elif b == "Developer":
        for p in phones:
            if p.split(" ")[0] == b:
                if "/" in p:
                    p_lst = p.split("/")
                    for series in p_lst[1:]:
                        sub_p = p_lst[0].replace(b, "").replace("5G", "").strip()
                        if series == "+":
                            phone = sub_p + series
                        else:
                            phone = sub_p + " " + series
                        if len(phone) > 0:
                            phone_dict[b].append(phone)
                    phone_dict[b].append(sub_p)
                else:
                    phone = p.replace(b, "").replace("5G", "").strip()
                    if len(phone) > 0:
                        print(phone)
                        phone_dict[b].append(phone)
                    phone_dict[b].append(phone)
    else:
        for p in phones:
            if p.split(" ")[0] == b:
                if "/" in p:
                    p_lst = p.split("/")
                    for series in p_lst[1:]:
                        sub_p = p_lst[0].replace(b, "").replace("5G", "").strip()
                        if series == "+":
                            phone = sub_p + series
                        else:
                            phone = sub_p + " " + series
                        if len(phone) > 0:
                            phone_dict[b].append(phone)
                    phone_dict[b].append(sub_p)
                else:
                    phone = p.replace(b, "").replace("5G", "").strip()
                    if len(phone) > 0:
                        phone_dict[b].append(phone)
                    phone_dict[b].append(phone)
phone_dict["Asus"].append("ROG Phone 2")
phone_dict.pop("Galaxy")
for k in phone_dict.keys():
    phone_dict[k] = sorted(phone_dict[k], reverse=True)

In [ ]:
import pprint
print(len(phone_dict))
pprint.pprint(phone_dict.keys())
pprint.pprint(phone_dict["Samsung"])

## iPhone

In [ ]:
url = "https://www.theiphonewiki.com/wiki/List_of_iPhones"
resp = requests.get(url)
soup = BeautifulSoup(resp.text, "html.parser")

table = soup.find("div", id="toc")
iphones = []
for span in table.select("ul li span"):
    if "iPhone" in str(span):
        if ("SE" in str(span)) and ("1" in str(span)):
            iphones.append("iPhone SE")
        elif ("SE" in str(span)) and ("2" in str(span)):
            iphones.append("iPhone SE2")
        else:
            iphones.append(span.text)
iphones = sorted(iphones, reverse=True)
phone_dict["iPhone"] = iphones
iphones.remove("iPhone")
iphones

In [ ]:
import pprint
print(len(phone_dict))
pprint.pprint(phone_dict)

In [ ]:
phone_brand_set = set()
for k in phone_dict.keys():
    for v in phone_dict[k]:
        for iphone in [ i.replace("iPhone", "").strip() for i in phone_dict["iPhone"] ]:
            if v == iphone:
                phone_brand_set.add(v)
iphone_matches = sorted([v for v in list(set([ i.replace("iPhone", "").strip() for i in phone_dict["iPhone"] ]) - phone_brand_set) if len(v) > 0], key=lambda x: len(x), reverse=True)
iphone_matches = [ i for i in iphone_matches if ("11" in i) or ("12" in i) or ("SE2" in i)]
iphone_matches.remove("11")
iphone_matches.remove("12")
iphone_matches

# crawl PTT

In [3]:
def get_last_page(url):
    ua = UserAgent()
    fakeua = ua.random
    headers = {"Origin": "https://www.ptt.cc",
            "Referer": "https://fonts.googleapis.com/",
            "sec-ch-ua-mobile": "?0",
            "User-Agent": fakeua}
    resp = requests.get(url, headers=headers)
    soup = BeautifulSoup(resp.text, "html.parser")
    previous = soup.find_all("a", class_="btn wide")[1]["href"]
    previous_page_num = int(re.findall(r"\d+", previous)[0])
    return previous_page_num
last_page = get_last_page("https://www.ptt.cc/bbs/MacShop/index.html")
last_page

20586

In [4]:
def crawl_macshop(links, page, sold, iphones):
    for link in links:
        url = "https://www.ptt.cc" + link
        # GET request from url and parse via BeautifulSoup
        ua = UserAgent()
        fakeua = ua.random
        headers = {"Origin": "https://www.ptt.cc",
                "Referer": "https://fonts.googleapis.com/",
                "sec-ch-ua-mobile": "?0",
                "User-Agent": fakeua}
        resp = requests.get(url, headers=headers)
        if "200" in str(resp):
            try:
                soup = BeautifulSoup(resp.text, "html.parser")

                # basic info: accout, board, title, created_at
                info = [ i.text.strip() for i in soup.findAll("span", class_="article-meta-value")]
                try:
                    spec = re.findall(r'規格.+\n', str(soup.find("div", id="main-content")))[0].strip()
                except:
                    spec = ""
                try:
                    title = info[2]

                    if len(title) == 0:
                        title = info[2]     
                except IndexError:
                    title = re.findall(r'型號.+\n', str(soup.find("div", id="main-content")))[0][2:].strip()

                title = " ".join(re.findall(r"[a-zA-Z0-9+]+", title))
                storage = None

                for iphone in iphones:
                    if iphone.lower().replace(" ", "") in title.lower().replace(" ", ""): 
                        for s in  ["32", "64", "128", "256", "512"]:
                            if (s in title) or (s in spec):
                                storage = int(s)
                                break
                        title = iphone
                        break

                if "iPhone" in title:
                    pass
                else:
                    for x in iphone_matches:
                        if x.lower().replace(" ", "") in title.lower().replace(" ", ""):
                            for iphone in iphones:
                                if x == phone.lower().replace(" ", ""):
                                    for s in  ["32", "64", "128", "256", "512"]:
                                        if (s in title) or (s in spec):
                                            storage = int(s)
                                            break
                                    title = iphone
                                    break
                            break

                try:
                    account = info[0]
                    created_at = datetime.strptime(info[3][3:].strip(), '%b %d %H:%M:%S %Y')
                except IndexError:
                    account = None
                    created_at = " ".join(re.findall(r'時間.+\n', str(soup.find("div", id="main-content")))[0].split(" ")[2:]).strip()
                    created_at = datetime.strptime(created_at, '%b %d %H:%M:%S %Y')

                # status
                try:
                    status = re.findall(r'型號.+\n', str(soup.find("div", id="main-content")))[0].strip()
                    if len(status) <= 5:
                        status = re.findall(r'型號.+規格', str(soup.find("div", id="main-content")).replace("\n", ""))[0]

                    if ("全新" in info[2]) or ("全新" in status):
                        new = True
                    elif ("近全新" in info[2]) or ("近全新" in status) or ("外觀全新" in status) or ("全新配件" in status) or ("全新耳機" in status):
                        new = False
                    else:
                        new = False
                except IndexError:
                    new = False

                # price
                try:
                    price = re.findall(r'售價.+\n', str(soup.find("div", id="main-content")))
                    price = price[0].strip().replace(",", "")
                    price = int(re.search(r'\d+', price).group())
                except AttributeError:
                    try:
                        price = re.findall(r'售價.+', str(soup.find("div", id="main-content")).replace("\n", "").replace(",", "").replace(" ", ""))[0].strip()
                        price = int(re.search(r'\d+', price).group())
                    except AttributeError:
                        price = None
                
                # items included
                try:
                    box = re.findall(r'盒裝.+\n', str(soup.find("div", id="main-content")))[0].strip()
                    if len(box) <= 5:
                        box = re.findall(r'盒裝配件.+售價', str(soup.find("div", id="main-content")).replace("\n", ""))[0]
                        box = box.replace(" ", "")[:-2]
                except IndexError:
                    box = None

                data.append([title, storage, price, new, sold, account, box, url, created_at, page, "MacShop"])
                raw_data.append([url, str(soup.find("body"))])
            except:
                error_links.append(url)

In [5]:
raw_data = []
data = []
error_links = []
for page in range(last_page, max_page, -1):
    ua = UserAgent()
    fakeua = ua.random
    headers = {"Origin": "https://www.ptt.cc",
            "Referer": "https://fonts.googleapis.com/",
            "sec-ch-ua-mobile": "?0",
            "User-Agent": fakeua}

    url = "https://www.ptt.cc/bbs/macshop/index{}.html".format(page)
    print(url)
    # GET request from url and parse via BeautifulSoup
    resp = requests.get(url, headers=headers)
    #resp.encoding = 'utf-8' # encoded with format utf-8 for chinese character
    index_soup = BeautifulSoup(resp.text, "html.parser")
    title = index_soup.findAll("div", class_="title")
    links = [ t.find("a")["href"] for t in title if ("販售" in str(t)) and ("已售" not in str(t)) and ("售出" not in str(t)) and ("iPhone" in str(t))]
    sold_links = [ t.find("a")["href"] for t in title if (("已售" in str(t)) or ("售出" in str(t))) and ("iPhone" in str(t))]

    crawl_macshop(links, page, sold=False, iphones=phone_dict["iPhone"])
    crawl_macshop(sold_links, page, sold=True, iphones=phone_dict["iPhone"])
    
    if page%100 == 0:
        db = Ptt()
        db.insert_post(raw_data, data)
        data = []
        raw_data = []
        print("insert into db for {}".format(page))

https://www.ptt.cc/bbs/macshop/index19099.html
https://www.ptt.cc/bbs/macshop/index19098.html
https://www.ptt.cc/bbs/macshop/index19097.html
https://www.ptt.cc/bbs/macshop/index19096.html
https://www.ptt.cc/bbs/macshop/index19095.html
https://www.ptt.cc/bbs/macshop/index19094.html
https://www.ptt.cc/bbs/macshop/index19093.html
https://www.ptt.cc/bbs/macshop/index19092.html
https://www.ptt.cc/bbs/macshop/index19091.html
https://www.ptt.cc/bbs/macshop/index19090.html
https://www.ptt.cc/bbs/macshop/index19089.html
https://www.ptt.cc/bbs/macshop/index19088.html
https://www.ptt.cc/bbs/macshop/index19087.html
https://www.ptt.cc/bbs/macshop/index19086.html
https://www.ptt.cc/bbs/macshop/index19085.html
https://www.ptt.cc/bbs/macshop/index19084.html
https://www.ptt.cc/bbs/macshop/index19083.html
https://www.ptt.cc/bbs/macshop/index19082.html
https://www.ptt.cc/bbs/macshop/index19081.html
https://www.ptt.cc/bbs/macshop/index19080.html
https://www.ptt.cc/bbs/macshop/index19079.html
https://www.p

KeyboardInterrupt: 

In [ ]:
print(len(error_links))

In [ ]:
import pprint
print(len(data))
data[0]

# Test

In [ ]:
ua = UserAgent()
fakeua = ua.random
headers = {"Origin": "https://www.ptt.cc",
        "Referer": "https://fonts.googleapis.com/",
        "sec-ch-ua-mobile": "?0",
        "User-Agent": fakeua}

def test_one_url(url, page=0):
    phone_brand_set = set()
    for k in phone_dict.keys():
        for v in phone_dict[k]:
            for iphone in [ i.replace("iPhone", "").strip() for i in phone_dict["iPhone"] ]:
                if v == iphone:
                    phone_brand_set.add(v)
    iphone_matches = sorted([v for v in list(set([ i.replace("iPhone", "").strip() for i in phone_dict["iPhone"] ]) - phone_brand_set) if len(v) > 0], key=lambda x: len(x), reverse=True)
    iphone_matches.remove("12")
    iphone_matches.remove("11")
        
    ua = UserAgent()
    fakeua = ua.random
    headers = {"Origin": "https://www.ptt.cc",
            "Referer": "https://fonts.googleapis.com/",
            "sec-ch-ua-mobile": "?0",
            "User-Agent": fakeua}
    resp = requests.get(url, headers=headers)
    if "200" in str(resp):
        soup = BeautifulSoup(resp.text, "html.parser")

        # basic info: accout, board, title, created_at
        info = [ i.text.strip() for i in soup.findAll("span", class_="article-meta-value")]
        try:
            spec = re.findall(r'規格.+\n', str(soup.find("div", id="main-content")))[0].strip()
        except:
            spec = ""
        try:
            title = info[2]

            if len(title) == 0:
                title = info[2]     
        except IndexError:
            title = re.findall(r'型號.+\n', str(soup.find("div", id="main-content")))[0][2:].strip()

        title = " ".join(re.findall(r"[a-zA-Z0-9+]+", title))
        storage = None

        for iphone in phone_dict["iPhone"]:
            if iphone.lower().replace(" ", "") in title.lower().replace(" ", ""): 
                for s in  ["32", "64", "128", "256", "512"]:
                    if (s in title) or (s in spec):
                        storage = int(s)
                        break
                title = iphone
                break

        if "iPhone" in title:
            pass
        else:
            for x in iphone_matches:
                if x.lower().replace(" ", "") in title.lower().replace(" ", ""):
                    for iphone in iphones:
                        if x == phone.lower().replace(" ", ""):
                            for s in  ["32", "64", "128", "256", "512"]:
                                if (s in title) or (s in spec):
                                    storage = int(s)
                                    break
                            title = iphone
                            break
                    break

        try:
            account = info[0]
            created_at = datetime.strptime(info[3][3:].strip(), '%b %d %H:%M:%S %Y')
        except IndexError:
            account = None
            created_at = " ".join(re.findall(r'時間.+\n', str(soup.find("div", id="main-content")))[0].split(" ")[2:]).strip()
            created_at = datetime.strptime(created_at, '%b %d %H:%M:%S %Y')

        # status
        try:
            status = re.findall(r'型號.+\n', str(soup.find("div", id="main-content")))[0].strip()
            if len(status) <= 5:
                status = re.findall(r'型號.+規格', str(soup.find("div", id="main-content")).replace("\n", ""))[0]

            if ("全新" in info[2]) or ("全新" in status):
                new = True
            elif ("近全新" in info[2]) or ("近全新" in status) or ("外觀全新" in status) or ("全新配件" in status) or ("全新耳機" in status):
                new = False
            else:
                new = False
        except IndexError:
            new = False

        # price
        try:
            price = re.findall(r'售價.+\n', str(soup.find("div", id="main-content")))
            price = price[0].strip().replace(",", "")
            price = int(re.search(r'\d+', price).group())
        except AttributeError:
            try:
                price = re.findall(r'售價.+', str(soup.find("div", id="main-content")).replace("\n", "").replace(",", "").replace(" ", ""))[0].strip()
                price = int(re.search(r'\d+', price).group())
            except AttributeError:
                price = None
        
        # items included
        try:
            box = re.findall(r'盒裝.+\n', str(soup.find("div", id="main-content")))[0].strip()
            if len(box) <= 5:
                box = re.findall(r'盒裝配件.+售價', str(soup.find("div", id="main-content")).replace("\n", ""))[0]
                box = box.replace(" ", "")[:-2]
        except IndexError:
            box = None

    print([title, storage, price, new, account, box, url, created_at, page])

In [ ]:
test_one_url("https://www.ptt.cc/bbs/MacShop/M.1620827180.A.619.html")

# Insert into db

In [ ]:
from crawler_model import Ptt
db = Ptt()
db.insert_post(raw_data, data)